In [22]:
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")


import sqlite3
import pandas as pd
import numpy as np
import nltk
import string
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.feature_extraction.text import CountVectorizer
from nltk.stem.porter import PorterStemmer

import re
# Tutorial about Python regular expressions: https://pymotw.com/2/re/
import string
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer

#from gensim.models import Word2Vec
#from gensim.models import KeyedVectors
import pickle

from tqdm import tqdm
import os


#from sklearn.cross_validation import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
#from sklearn.cross_validation import cross_val_score
from collections import Counter
#from sklearn import cross_validation
from sklearn.model_selection import cross_validate
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve, roc_auc_score,f1_score,auc

from scipy import sparse
import scipy
from scipy.sparse import csr_matrix
import json
preprocessed_reviews = []

In [23]:
with open('insta.json', 'r') as f:
    data = json.load(f)
final_data = pd.io.json.json_normalize(data['data'])
final_data

,attribution,caption.created_time,caption.from.full_name,caption.from.id,caption.from.profile_picture,caption.from.username,caption.id,caption.text,comments.count,created_time,...,location.longitude,location.name,tags,type,user.full_name,user.id,user.profile_picture,user.username,user_has_liked,users_in_photo
0,None,1557500555,Emma Foster,13335662569,https://scontent.cdninstagram.com/vp/e7a67cbe4...,emmafoster4490,17929753903292079,"Some days are just bad days, that's all. You h...",0,1557500555,...,151.207970,"Sydney, Australia","[depression, sad, saddepression, sadedit, brok...",image,Emma Foster,13335662569,https://scontent.cdninstagram.com/vp/e7a67cbe4...,emmafoster4490,False,[]
1,None,1557500066,Emma Foster,13335662569,https://scontent.cdninstagram.com/vp/e7a67cbe4...,emmafoster4490,17883548272342012,Grief is never something you get over. You don...,0,1557500066,...,152.144887,"Nelson Bay Beach, New South Wales, Australia","[sadvideos, omg, hurt, deppressingedits, lonel...",image,Emma Foster,13335662569,https://scontent.cdninstagram.com/vp/e7a67cbe4...,emmafoster4490,False,[]
2,None,1557499693,Emma Foster,13335662569,https://scontent.cdninstagram.com/vp/e7a67cbe4...,emmafoster4490,17880283147351161,Let my soul smile through my heart and my hear...,0,1557499693,...,151.207970,"Sydney, Australia","[sadposts, waitingforgooddays, lifequotes, sad...",image,Emma Foster,13335662569,https://scontent.cdninstagram.com/vp/e7a67cbe4...,emmafoster4490,False,[]
3,None,1557499580,Emma Foster,13335662569,https://scontent.cdninstagram.com/vp/e7a67cbe4...,emmafoster4490,18035147494188621,"Heavy hearts, like heavy clouds in the sky, ar...",1,1557499580,...,NaN,NaN,"[chronicpain, trauma, sad, peace, stress, inst...",image,Emma Foster,13335662569,https://scontent.cdninstagram.com/vp/e7a67cbe4...,emmafoster4490,False,"[{'user': {'username': 'chocolate_user'}, 'pos..."
4,None,1557499500,Emma Foster,13335662569,https://scontent.cdninstagram.com/vp/e7a67cbe4...,emmafoster4490,18063639889015231,"When you wake up each morning, you can choose ...",0,1557499500,...,NaN,NaN,"[needhappiness, sorrow, sadmusings, scarynight...",image,Emma Foster,13335662569,https://scontent.cdninstagram.com/vp/e7a67cbe4...,emmafoster4490,False,"[{'user': {'username': 'chocolate_user'}, 'pos..."
5,None,1557499329,Emma Foster,13335662569,https://scontent.cdninstagram.com/vp/e7a67cbe4...,emmafoster4490,17906319535305303,There is a sacredness in tears. They are not t...,1,1557499329,...,NaN,NaN,"[sorrow, anxiety, sadness, deepthoughts, grief...",image,Emma Foster,13335662569,https://scontent.cdninstagram.com/vp/e7a67cbe4...,emmafoster4490,False,[]
6,None,1557497248,Emma Foster,13335662569,https://scontent.cdninstagram.com/vp/e7a67cbe4...,emmafoster4490,17890858519325827,"People keep telling me that life goes on, but ...",0,1557497248,...,152.400000,"Brooyar, Queensland, Australia","[chronicpain, trauma, sad, peace, stress, inst...",image,Emma Foster,13335662569,https://scontent.cdninstagram.com/vp/e7a67cbe4...,emmafoster4490,False,[]
7,None,1557496801,Emma Foster,13335662569,https://scontent.cdninstagram.com/vp/e7a67cbe4...,emmafoster4490,18064781146004313,"I miss me. The old me, the happy me, the brigh...",0,1557496801,...,NaN,NaN,"[chronicpain, trauma, sad, peace, stress, inst...",image,Emma Foster,13335662569,https://scontent.cdninstagram.com/vp/e7a67cbe4...,emmafoster4490,False,[]
8,None,1557496675,Emma Foster,13335662569,https://scontent.cdninstagram.com/vp/e7a67cbe4...,emmafoster4490,17987240635225351,"You smile, but you wanna cry. You talk, but yo...",2,1557496675,...,138.477155,"Semaphore South Beach, Adelaide, South Australia","[cry, lonely, depression, sad, love, mood, bro...",image,Emma Foster,13335662569,https://scontent.cdninstagram.com/vp/e7a67cbe4...,emmafoster4490,False,[]
9,None,1557495396,Emma Foster,13335662569,https://scontent.cdninstagram.com/vp/e7a67cbe4...,emmafoster4490,17842941313449295,"My moods don’t just swing-they bounce, pivot, ...",1,1557495396

In [24]:
df = pd.DataFrame(final_data)
df = df[["caption.text"]]
df

,caption.text
0,"Some days are just bad days, that's all. You h..."
1,Grief is never something you get over. You don...
2,Let my soul smile through my heart and my hear...
3,"Heavy hearts, like heavy clouds in the sky, ar..."
4,"When you wake up each morning, you can choose ..."
5,There is a sacredness in tears. They are not t...
6,"People keep telling me that life goes on, but ..."
7,"I miss me. The old me, the happy me, the brigh..."
8,"You smile, but you wanna cry. You talk, but yo..."
9,"My moods don’t just swing-they bounce, pivot, ..."


In [25]:
import re
from bs4 import BeautifulSoup

def decontracted(phrase):
    # specific
    phrase = re.sub(r"won't", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)

    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    return phrase

for i in range(20):
    sent_0 = df['caption.text'].values[i]
    # Remove all the hashtags from captions
    sent_0 = re.sub(r'#\w+', '', sent_0)
    # Remove urls from text python: https://stackoverflow.com/a/40823105/4084039
    sent_0 = re.sub(r"http\S+", "", sent_0)
    # https://stackoverflow.com/questions/16206380/python-beautifulsoup-how-to-remove-all-tags-from-an-element
    soup = BeautifulSoup(sent_0, 'lxml')
    text = soup.get_text()
    text = decontracted(text)
    # Remove words with numbers python: https://stackoverflow.com/a/18082370/4084039
    text = re.sub("\S*\d\S*", "", text).strip()
    # Remove special character: https://stackoverflow.com/a/5843547/4084039
    text = re.sub('[^A-Za-z0-9]+', ' ', text)
    print(text)
    print("="*50)

Some days are just bad days that is all You have to experience sadness to know happiness and I remind myself that not every day is going to be a good day that is just the way it is 
Grief is never something you get over You do not wake up one morning and say I have conquered that now I am moving on It is something that walks beside you every day And if you can learn how to manage it and honour the person that you miss you can take something that is incredibly sad and have some form of positivity 
Let my soul smile through my heart and my heart smile through my eyes that I may scatter rich smiles in sadhearts Some days are just bad days that is all You have to experience sadness to know happiness and I remind myself that not every day is going to be a good day that is just the way it is 
Heavy hearts like heavy clouds in the sky are best relieved by the letting of a little water 
When you wake up each morning you can choose to be happy or choose to be sad Unless some terrible catastroph

In [26]:
# https://gist.github.com/sebleier/554280
# we are removing the words from the stop words list: 'no', 'nor', 'not'
# <br /><br /> ==> after the above steps, we are getting "br br"
# we are including them into stop words list
# instead of <br /> if we have <br/> these tags would have revmoved in the 1st step

stopwords= set(['br', 'the', 'i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've",\
            "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', \
            'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their',\
            'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', \
            'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', \
            'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', \
            'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after',\
            'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further',\
            'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more',\
            'most', 'other', 'some', 'such', 'only', 'own', 'same', 'so', 'than', 'too', 'very', \
            's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', \
            've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn',\
            "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn',\
            "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", \
            'won', "won't", 'wouldn', "wouldn't"])

In [27]:
# Combining all the above stundents 
from tqdm import tqdm
#preprocessed_reviews = []
# tqdm is for printing the status bar
for sentance in tqdm(df['caption.text'].values):
    sentance = re.sub(r'#\w+', '', sentance)
    sentance = re.sub(r"http\S+", "", sentance)
    sentance = BeautifulSoup(sentance, 'lxml').get_text()
    sentance = decontracted(sentance)
    sentance = re.sub("\S*\d\S*", "", sentance).strip()
    sentance = re.sub('[^A-Za-z]+', ' ', sentance)
    # https://gist.github.com/sebleier/554280
    sentance = ' '.join(e.lower() for e in sentance.split() if e.lower() not in stopwords)
    preprocessed_reviews.append(sentance.strip())

100%|██████████| 20/20 [00:00<00:00, 3990.21it/s]


In [143]:
preprocessed_reviews[0]

'days bad days experience sadness know happiness remind not every day going good day way'

In [28]:
from sklearn.feature_extraction.text import CountVectorizer
#BoW
count_vect_bow = CountVectorizer() #in scikit-learn
count_vect_bow.fit(preprocessed_reviews)
print("some feature names ", count_vect_bow.get_feature_names()[:10])
print('='*50)

final_counts = count_vect_bow.transform(preprocessed_reviews)
print(final_counts)
print("the type of count vectorizer ",type(final_counts))
print("the shape of out text BOW vectorizer ",final_counts.get_shape())
print("the number of unique words ", final_counts.get_shape()[1])


sparse.save_npz("bowmatrix.npz",final_counts)

some feature names  ['aching', 'also', 'around', 'attempt', 'bad', 'bear', 'beside', 'best', 'bounce', 'bright']
  (0, 4)	1
  (0, 27)	2
  (0, 28)	2
  (0, 38)	1
  (0, 40)	1
  (0, 54)	1
  (0, 56)	1
  (0, 61)	1
  (0, 79)	1
  (0, 108)	1
  (0, 135)	1
  (0, 142)	1
  (0, 183)	1
  (1, 6)	1
  (1, 20)	1
  (1, 27)	1
  (1, 38)	1
  (1, 49)	1
  (1, 52)	1
  (1, 59)	1
  (1, 69)	1
  (1, 75)	1
  (1, 81)	1
  (1, 91)	1
  (1, 99)	1
  :	:
  (16, 39)	1
  (16, 44)	1
  (16, 54)	1
  (16, 58)	1
  (16, 73)	1
  (16, 113)	1
  (16, 157)	1
  (16, 189)	1
  (17, 41)	1
  (17, 43)	1
  (17, 48)	1
  (17, 68)	1
  (17, 153)	1
  (17, 175)	1
  (18, 2)	1
  (18, 37)	1
  (18, 48)	1
  (18, 60)	1
  (18, 72)	1
  (18, 78)	1
  (18, 88)	1
  (18, 98)	1
  (18, 104)	1
  (18, 112)	1
  (18, 185)	1
the type of count vectorizer  <class 'scipy.sparse.csr.csr_matrix'>
the shape of out text BOW vectorizer  (20, 190)
the number of unique words  190


In [29]:
#bi-gram, tri-gram and n-gram

#removing stop words like "not" should be avoided before building n-grams
# count_vect = CountVectorizer(ngram_range=(1,2))
# please do read the CountVectorizer documentation http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html

# you can choose these numebrs min_df=10, max_features=5000, of your choice
count_vect = CountVectorizer(ngram_range=(1,2), min_df=1, max_features=5000)
final_bigram_counts = count_vect.fit_transform(preprocessed_reviews)
print("the type of count vectorizer ",type(final_bigram_counts))
print("the shape of out text BOW vectorizer ",final_bigram_counts.get_shape())
print("the number of unique words including both unigrams and bigrams ", final_bigram_counts.get_shape()[1])


sparse.save_npz("bowbigramsmatrix.npz",final_bigram_counts)

the type of count vectorizer  <class 'scipy.sparse.csr.csr_matrix'>
the shape of out text BOW vectorizer  (20, 415)
the number of unique words including both unigrams and bigrams  415


In [30]:
tf_idf_vect = TfidfVectorizer(ngram_range=(1,2), min_df=1)
tf_idf_vect.fit(preprocessed_reviews)
print("some sample features(unique words in the corpus)",tf_idf_vect.get_feature_names()[0:10])
print('='*50)

final_tf_idf = tf_idf_vect.transform(preprocessed_reviews)
print("the type of count vectorizer ",type(final_tf_idf))
print("the shape of out text TFIDF vectorizer ",final_tf_idf.get_shape())
print("the number of unique words including both unigrams and bigrams ", final_tf_idf.get_shape()[1])

sparse.save_npz("tfidfmatrix.npz",final_tf_idf)

some sample features(unique words in the corpus) ['aching', 'aching say', 'also', 'also hard', 'around', 'around minute', 'attempt', 'attempt conceal', 'bad', 'bad days']
the type of count vectorizer  <class 'scipy.sparse.csr.csr_matrix'>
the shape of out text TFIDF vectorizer  (20, 415)
the number of unique words including both unigrams and bigrams  415


In [31]:
# SingleFold function computes Hyperparameter 'alpha' where CV accuracy is highest in single fold in the range of 1 to 30

def mnbSingleFold(X_tr,y_tr,X_cv,y_cv):   
    auc_accuracy={}
    auc_accuracy_tr={}
    auc_train=[]
    auc_cv=[]
    pred_train=pd.DataFrame()
    alpha=np.array([0.00001,0.0001,0.001,0.01,0.1,1.0,10.0,100.0,1000.0,10000.0])
    for i in alpha:
            
            # instantiate learning model (for alpha ranging from 10e-4 to 10e+4)
            mnb = MultinomialNB(alpha=i)

            # fitting the model on train data
            mnb.fit(X_tr, y_tr)

            # predict the response on the crossvalidation data
            pred_cv = mnb.predict_log_proba(X_cv)
            # predict the response on the train data
            pred_train = mnb.predict_log_proba(X_tr)
            
            # evaluate CV & train AUC score
            acc = roc_auc_score(y_cv, pred_cv[:,1]) *float(100)
            acc_train = roc_auc_score(y_tr, pred_train[:,1]) *float(100)
            auc_accuracy_tr[i]=acc_train
            auc_accuracy[i]=acc
    auc_train=list(auc_accuracy_tr.values())
    auc_cv=list(auc_accuracy.values())
    plt.title("Hyperparameter plotting against auc score on train & cv data")
    plt.plot(alpha, auc_train, "b--", label="TrainAUC")
    plt.plot(alpha, auc_cv, "g-", label="ValidationAUC")
    plt.ylabel("AUC Score")
    plt.xlabel("Hyperparameter Alpha")
    plt.legend()
    plt.show()
    # We will be choosing 'alpha' where AUC score for CV is highest 
    maxalpha=max(auc_accuracy, key=auc_accuracy.get) 
    print('\nHighest AUC Score for alpha = %d is %d%% using Multinomial NB' % (maxalpha ,auc_accuracy[maxalpha]))
    return(maxalpha,pred_train)

In [104]:
# Fuction to compute alpha value
def mnbNFold(X_train, y_train):
    
    alpha_values=np.array([0.00001,0.0001,0.001,0.01,0.1,1.0,10.0,100.0,1000.0,10000.0])
        

    # empty list that will hold cv scores
    cv_scores = []

    # perform 10-fold cross validation
    for alpha in alpha_values:
        mnb = MultinomialNB(alpha = alpha)
        scores = cross_val_score(mnb, X_train, y_train, cv = 10, scoring = 'roc_auc')
        cv_scores.append(scores.mean())

    # changing to misclassification error
    MSE = [1 - x for x in cv_scores]

    # determining best alpha
    optimal_alpha = alpha_values[MSE.index(min(MSE))]
    print('\nThe optimal number of alpha is %d.' % optimal_alpha)

    # plot misclassification error vs alpha
    plt.plot(alpha_values, MSE, marker = '*') 

    #for xy in zip(alpha_values, np.round(MSE,3)):
        #plt.annotate('(%s, %s)' % xy, xy=xy, textcoords='data')
    plt.title("Misclassification Error vs alpha")
    plt.xlabel('value of alpha')
    plt.ylabel('Misclassification Error')
    plt.show()

    #print("the misclassification error for each value of alpha is : ", np.round(MSE,3))
    return optimal_alpha

In [36]:
# split the dataset into train and  test in the ratio 70:30
def splitData(X,y):
    X_1, X_test, y_1, y_test = cross_validation.train_test_split(X, y, test_size=0.3, random_state=0)
    # Again split the train data set into cross validation train and cross validation test with 70:30 ratio
    X_tr, X_cv, y_tr, y_cv = cross_validation.train_test_split(X_1, y_1, test_size=0.3)
    print("So 100000 datapoints we have splitted in 3 categories train,cv and test and below are the respective matrix dimensions: ")
    print("Train Data Size: {0}{1} " .format(X_tr.shape,y_tr.shape))
    print("CV Data Size: {0}{1} " .format(X_cv.shape,y_cv.shape))
    print("Test Data Size: {0}{1} " .format(X_test.shape,y_test.shape))
    return(X_tr, X_cv,X_test,X_1,y_tr, y_cv,y_test,y_1)

In [45]:
X_tr_bow,X_cv_bow,X_test_bow,X_1_bow,y_tr, y_cv,y_test,y_1=splitData(final_counts,df['caption.text'])

NameError: name 'cross_validation' is not defined

In [41]:
import nltk
import sklearn

print('The nltk version is {}.'.format(nltk.__version__))
print('The scikit-learn version is {}.'.format(sklearn.__version__))

The nltk version is 3.4.
The scikit-learn version is 0.20.3.
